In [61]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
from datetime import datetime
from fbprophet import Prophet
import pickle

In [2]:
df = pd.read_csv("./data/ts_processed.csv",parse_dates=[0])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674116 entries, 0 to 674115
Data columns (total 5 columns):
drp_dtm         674116 non-null datetime64[ns]
drp_yyyymmdd    674116 non-null int64
drp_hrmi        674116 non-null float64
drp_loc         674116 non-null object
trips_count     674116 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 25.7+ MB


In [7]:
df.head()

,drp_dtm,drp_yyyymmdd,drp_hrmi,drp_loc,trips_count
0,2016-01-01,20160101,0.0,-73.93_40.71,1
1,2016-01-01,20160101,0.0,-73.94_40.83,1
2,2016-01-01,20160101,0.0,-73.94_40.85,1
3,2016-01-01,20160101,0.0,-73.95_40.82,1
4,2016-01-01,20160101,0.0,-73.97_40.74,1


In [5]:
df.drp_yyyymmdd.min()

20160101

In [6]:
df.drp_yyyymmdd.max()

20160701

Train using 5 months data for all each of the location

In [14]:
df[(df['drp_dtm'] >= datetime(2016,1,1)) & (df['drp_dtm'] < datetime(2016,6,1)) & (df['drp_loc'] == '-73.94_40.83')][['drp_dtm','trips_count']].head()

,drp_dtm,trips_count
1,2016-01-01 00:00:00,1
30,2016-01-01 00:15:00,1
85,2016-01-01 00:30:00,2
141,2016-01-01 00:45:00,1
211,2016-01-01 01:00:00,1


In [128]:
# Function to train Prophet model for the given location.
# Returns the trained model
def train_for(a_drp_loc):    
    pf = df[(df['drp_dtm'] >= datetime(2016,1,1)) & (df['drp_dtm'] < datetime(2016,6,1)) & (df['drp_loc'] == a_drp_loc)][['drp_dtm','trips_count']]
    pf = pf.rename({'drp_dtm':'ds', 'trips_count':'y'},axis=1)
    pf['cap'] = 25 # We set 25 as the maximum number of trips possible in that time slot in that location
    m = Prophet(growth='logistic')
    m.fit(pf)
    return m

In [129]:
# Train for locations that have at least 1000 trips count
g = df.groupby('drp_loc').filter(lambda r: r['trips_count'].sum() >= 1000).groupby('drp_loc')
models=dict()
cnt = 0
for k, gp in g:
    cnt += 1
    print ('Training: ' + str(cnt) + ' k=' + k)
    models[k] = train_for(k)

INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Training: 1 k=-73.78_40.64


/anaconda3/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Training: 2 k=-73.78_40.65


INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


...

Training: 108 k=-74.0_40.76


INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Training: 109 k=-74.0_40.77


INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Training: 110 k=-74.18_40.69


In [130]:
models

{'-73.78_40.64': <fbprophet.forecaster.Prophet at 0x1c0b9f4668>,
 '-73.78_40.65': <fbprophet.forecaster.Prophet at 0x1c0b880390>,
 '-73.79_40.64': <fbprophet.forecaster.Prophet at 0x1c0ad25198>,
 '-73.79_40.65': <fbprophet.forecaster.Prophet at 0x1c0ac6bba8>,
 '-73.86_40.77': <fbprophet.forecaster.Prophet at 0x1c0b709080>,
 '-73.87_40.77': <fbprophet.forecaster.Prophet at 0x1c0d26bb70>,
 '-73.89_40.75': <fbprophet.forecaster.Prophet at 0x1c1d57fb38>,
 '-73.91_40.76': <fbprophet.forecaster.Prophet at 0x1c1d57fcc0>,
 '-73.91_40.77': <fbprophet.forecaster.Prophet at 0x1c0ad98a20>,
 '-73.92_40.74': <fbprophet.forecaster.Prophet at 0x1c1df96eb8>,
 '-73.92_40.76': <fbprophet.forecaster.Prophet at 0x1c1cf58a20>,
 '-73.92_40.77': <fbprophet.forecaster.Prophet at 0x1c0d7067f0>,
 '-73.93_40.7': <fbprophet.forecaster.Prophet at 0x1c1dfb1d68>,
 '-73.93_40.76': <fbprophet.forecaster.Prophet at 0x1c1cf58550>,
 '-73.93_40.77': <fbprophet.forecaster.Prophet at 0x1c1d41c3c8>,
 '-73.93_40.8': <fbprophet

Serialize the trained models, so that it can be used for prediction

In [131]:
with open('./data/models.pkl','wb') as of:
    pickle.dump(models, of)

Create a function that reads the serialized models and can predict

In [133]:
with open('./data/models.pkl','rb') as fi:
    mds = pickle.load(fi)
def predict_for(drp_loc, drp_time):
    pdf = pd.DataFrame({'ds': [drp_time], 'cap' : 25.0})
    try:
        fc = mds[drp_loc].predict(pdf)
        #print (fc)
    except KeyError:
        return 'Location not trained for prediction'
    return 'Predicted trips: ' + str(int(round(fc.yhat[0],0)))

In [136]:
predict_for('-73.95_40.78',datetime(2016,6,1,0,0))

'Predicted trips: 3'

We have created a function that predict the number of trips for a given location and time